In [ ]:
# QGIS 분석을 위한 위도경도 추출 코드(카카오 API 활용)

In [1]:
# 필요한 패키지 로드
import pandas as pd
import numpy as np
import json
import requests

In [41]:
## csv파일 열기 (encoding 옵션은 종류에 따라 수정)
file = pd.read_csv('../data/10.세종시_오피스텔(전월세)_실거래가_주소.csv', encoding = 'utf-8')

In [44]:
addr = '세종특별자치시 보람동 743'
url='https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
headers = {"Authorization": "KakaoAK 057873079626a592436710b8c7b612e8"}
result = json.loads(str(requests.get(url, headers=headers).text))
# match_first = result['documents'][0]['x']

print(result)
# print(match_first)

{'documents': [{'address': {'address_name': '세종특별자치시 보람동 743', 'b_code': '3611010300', 'h_code': '3611056000', 'main_address_no': '743', 'mountain_yn': 'N', 'region_1depth_name': '세종특별자치시', 'region_2depth_name': '', 'region_3depth_h_name': '보람동', 'region_3depth_name': '보람동', 'sub_address_no': '', 'x': '127.28911728793', 'y': '36.4783430775605'}, 'address_name': '세종특별자치시 보람동 743', 'address_type': 'REGION_ADDR', 'road_address': {'address_name': '세종특별자치시 한누리대로 2135', 'building_name': '스타힐타워Ⅰ', 'main_building_no': '2135', 'region_1depth_name': '세종특별자치시', 'region_2depth_name': '', 'region_3depth_name': '보람동', 'road_name': '한누리대로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.289160385804', 'y': '36.4783499120552', 'zone_no': '30150'}, 'x': '127.28911728793', 'y': '36.4783430775605'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [45]:
# 만약 찾지 못할 경우, 이전 위경도로 저장되므로 중복처리 한번 해줘야 함

for i in range(len(file)):
    # ---이부분은 생략가능(얼마나 진행됬는지 확인)---
    if (i%500 == 0):
        print(str(i)+" rows done")
    # ------------------------------------------------
    try:
        match_first= {'x': 0, 'y': 0}
        addr = file['주소'][i]
        url='https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
        headers = {"Authorization": "KakaoAK 057873079626a592436710b8c7b612e8"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        

        ## 데이터 추가
        file['위도'][i] = match_first['y']
        file['경도'][i] = match_first['x']
        
#     except IndexError:
#         pass
    
#     except JSONDecodeError:
#         pass 
    
    except Exception as ex:
        pass

0 rows done


<ipython-input-45-5f59f2514b05>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['위도'][i] = match_first['y']
<ipython-input-45-5f59f2514b05>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['경도'][i] = match_first['x']


500 rows done
1000 rows done


In [47]:
file.to_csv("주소변환_오피스텔전월세.csv", encoding = "utf-8")